In [21]:
#import modules
import pandas as pd
import xlwings as xw
from rapidfuzz import fuzz, process
from pathlib import Path
from docxtpl import DocxTemplate
import os
import datetime
import re
from docx import Document
from docx.shared import Inches
from pathlib import Path
import rapidfuzz
import numpy as np
from rapidfuzz import fuzz
from rapidfuzz.process import extractOne
import datetime 
from pathlib import Path
from docxtpl import DocxTemplate
from docx import Document
from docx.shared import Pt
import subprocess
from tkinter import filedialog
from tkinter import Tk
import win32com.client as win32

In [22]:
#A GENERIC FUNCTIONS

#A.1 Closest Match btw string and dataframe Headers
from rapidfuzz import fuzz, process

def closest_match_df(string, df):
    column_names = df.columns.tolist()
    
    # Using rapidfuzz process to find the closest match
    closest_match = process.extractOne(string, column_names)
    
    # Getting the column index of the closest match
    closest_match_index = column_names.index(closest_match[0])
            
    return closest_match_index


#A.2 Find Closest Match btw a query and list of dictionaries but pass cutoff
def find_closest_match(query, list_of_dicts, score_cutoff=90):
    best_match = None
    best_score = 0
    best_result = None
    
    for dict_item in list_of_dicts:
        if 'Field Description' in dict_item:  # Make sure 'Field Description' key exists
            score = fuzz.ratio(query, dict_item['Field Description'])
            if score > best_score:
                best_score = score
                best_match = dict_item['Field Description']
                best_result = dict_item['Result'] if 'Result' in dict_item else None

    if best_score >= score_cutoff:
        return best_match, best_result
    else:
        return 'Not Found', None

#A.3 Find Closest Folder
def find_client_folder(Ref_No, folder_prefix, folder_1_suffix, folder_2_suffix, folder_3_suffix):
    folder_names = [folder_1_suffix, folder_2_suffix, folder_3_suffix]
    for folder_suffix in folder_names:
        folder_path = os.path.join(folder_prefix, folder_suffix)
        for subfolder in os.listdir(folder_path):
            if Ref_No in subfolder:
                return os.path.join(folder_path, subfolder)
    return None

#A.4 Create Folder
def create_folder(Last_Name, First_Name, Ref_No):
    root = tk.Tk()
    root.withdraw()
    folder_path = filedialog.askdirectory()
    new_folder_name = f"{Last_Name}.{First_Name}.[{Ref_No}]"
    new_folder_path = os.path.join(folder_path, new_folder_name)
    os.mkdir(new_folder_path)
    return new_folder_path

#A.5 Find or Create Folder
def find_or_create_folder(Last_Name, First_Name, Ref_No, folder_prefix, folder_1_suffix, folder_2_suffix, folder_3_suffix):
    client_folder_path = find_client_folder(Ref_No, folder_prefix, folder_1_suffix, folder_2_suffix, folder_3_suffix)
    if client_folder_path is None:
        #print("Unable to find the client folder.")
        #print("1: Create a folder")
        #print("2: Pick a current folder")
        option = input("Choose an option: ")
        if option == "1":
            client_folder_path = create_folder(Last_Name, First_Name, Ref_No)
        elif option == "2":
            root = tk.Tk()
            root.withdraw()
            client_folder_path = filedialog.askdirectory()
    return client_folder_path

#A.6 Get Subdirectories
def get_subdirectories(directory):
    return [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]

#A.8 
def remove_text_in_brackets(df):
    for col in df.columns:
        if 'Company' in col:
            if df[col].iloc[0] not in [0, '']:
                df[col] = df[col].str.replace(r'\[.*\]', '')
    return df

#A.9 Ordinal
def ordinal(n):
    suffix = ["th", "st", "nd", "rd"]
    if n < 0:
        return str(n)
    elif n % 100 in [11, 12, 13]:  # special case
        return str(n) + "th"
    else:
        return str(n) + suffix[n % 10 if n % 10 < 4 else 0]



In [23]:
#1 Define Sheets, Tables, dfs
def defineshtb():
    # 1.1: Retrieve currently active workbook and its path
    currentwb = xw.books.active
    original_excel_path = currentwb.fullname    
    
    # 1.2: Define current timestamp and parent folder
    current_datetime = datetime.datetime.now().strftime("%Y%m%d%H%M")
    currentpath = Path(original_excel_path)
    parent_folder = currentpath.parent    
    #print(parent_folder)
          
    # 1.5 Define Sheets
    shcalcs_pc = currentwb.sheets['calcs_pc']

    shCalcs = currentwb.sheets['Calcs']
    shLists = currentwb.sheets['Lists']
    shDDCTemplates = currentwb.sheets['DDCTemplates']
    shInvestTemplates = currentwb.sheets['InvestTemplates']
    shSTTemplates = currentwb.sheets['STTemplates']
    shMedGapTemplates = currentwb.sheets['MedGapTemplates']
    shStart = currentwb.sheets['Start']
    shhdr_col_static = currentwb.sheets['hdr_col_static']
    shUserMapping = currentwb.sheets['UserMapping']
    
    # 1.16 Convert each table range to a DataFrame
    tbTemplList = shLists.tables['tbTemplList']
    tbFieldList = shLists.tables['tbFieldList']
    tb_calc_pc = shCalcs.tables['tb_calc_pc']
    
    df_tbTemplList = pd.DataFrame(tbTemplList.range.value[1:], columns=tbTemplList.range.value[0])
    df_tbFieldList = pd.DataFrame(tbFieldList.range.value[1:], columns=tbFieldList.range.value[0])
    df_tb_calc_pc = pd.DataFrame(tb_calc_pc.range.value[1:], columns=tb_calc_pc.range.value[0])
    
    df_import_pc = shStart.range('A1').options(pd.DataFrame, expand='table', header=False).value
    header = df_import_pc.iloc[0]
    df_import_pc = df_import_pc[1:]
    df_import_pc.columns = header
    # Reset the index
    df_import_pc = df_import_pc.reset_index(drop=True)
    #print(df_import_pc.head())
    
    df_hdr_col_static = shhdr_col_static.range("A1").options(pd.DataFrame, expand="table").value
    
    return (
        current_datetime, shUserMapping, parent_folder, shhdr_col_static, df_hdr_col_static, df_tb_calc_pc, df_import_pc, currentwb, shcalcs_pc,
        shCalcs, shLists, shDDCTemplates, shInvestTemplates, shSTTemplates, shMedGapTemplates, shStart,
        tbTemplList, tbFieldList, df_tbTemplList, df_tbFieldList
    )

In [24]:
#1.2
def defineusermapping(shUserMapping, currentwb, shStart, shCalcs):
    Ref_No = shStart.range('A2').value
    user = xw.apps.active.api.UserName
    user_column = shUserMapping.range('A:A').options(transpose=True).value   
    # Find the client folder prefix corresponding to the current user
    user_index = next((i for i, name in enumerate(user_column) if name == user), None)
   
   # Retrieve the client folder prefix, user_excel_template_suffix and user_word_template_suffix if found, otherwise use default values
    if user_index is not None:
       folder_prefix = shUserMapping.range(f'B{user_index+1}').value
       user_word_template_suffix = shUserMapping.range(f'C{user_index+1}').value
       folder_1_suffix = shUserMapping.range(f'D{user_index+1}').value
       folder_2_suffix = shUserMapping.range(f'E{user_index+1}').value
       folder_3_suffix = shUserMapping.range(f'F{user_index+1}').value
       shUserMapping.range('F2').value = folder_prefix
    else:
       folder_prefix = 'default_prefix'
       user_word_template_suffix = 'default_word_suffix'
       folder_1_suffix = 'default_suffix_1'
       folder_2_suffix = 'default_suffix_2'
       folder_3_suffix = 'default_suffix_3'    

    # Prepare the user_word_search_string
    user_word_search_string = folder_prefix + user_word_template_suffix

    # Check if the file exists
    if not os.path.isfile(user_word_search_string):
        # If not, prompt the user to select a template
        root = Tk()
        root.withdraw()  # we don't want a full GUI, so keep the root window from appearing
        user_word_template = filedialog.askopenfilename()  # show an "Open" dialog box and return the path to the selected file
        root.destroy()
    else:
        user_word_template = user_word_search_string  # Use the default template
        
    shCalcs.range('E2').value = user_word_template

    return Ref_No, folder_prefix, user_word_template, folder_1_suffix, folder_2_suffix, folder_3_suffix


In [25]:
#2 Define and manage DDC Contracts
def No_of_Contracts(df_import_pc, currentwb, shDDCTemplates, shLists):
    # assume df is your DataFrame with multiple columns
    df_Death_RC_Contracts = df_import_pc.loc[:, df_import_pc.columns.str.contains('rc', case=False) & df_import_pc.columns.str.contains('category', case=False)]
    df_Death_RC_Contracts.head()
    count_Death_in_rows = df_Death_RC_Contracts.apply(lambda row: row.astype(str).str.contains('death', case=False).sum(), axis=1).sum()
    # List the column header names of the columns where the values contain "death"
    columns_with_death = [column for column in df_Death_RC_Contracts.columns if any(df_Death_RC_Contracts[column].astype(str).str.contains('death', case=False))]

    # Extract the number from the first column header name
    first_rc_containing_death = re.search(r'RC(\d+)', columns_with_death[0]).group(1)

    # Extract the number from the second column header name if count_Death_in_rows > 1
    if len(columns_with_death) > 1:
        second_rc_containing_death = re.search(r'RC(\d+)', columns_with_death[1]).group(1)
    else:
        second_rc_containing_death = "NA"\
            
    # Extract the number from the 3 column header name if count_Death_in_rows > 1
    if len(columns_with_death) > 3:
        third_rc_containing_death = re.search(r'RC(\d+)', columns_with_death[2]).group(1)
    else:
        third_rc_containing_death = "NA"
        
    df_IP_RC_Contracts = df_import_pc.loc[:, df_import_pc.columns.str.contains('rc', case=False) & df_import_pc.columns.str.contains('p2', case=False)]
    
    # Check if any valid numbers other than 0.0 exist in the DataFrame
    has_income_protection = any(df_IP_RC_Contracts.iloc[:, 1:].apply(lambda x: any(pd.to_numeric(x, errors='coerce').fillna(0) != 0.0), axis=1))  
     
    if count_Death_in_rows > 0:
        if count_Death_in_rows == 1:
            ddc_template = 'tbDDC1'
        elif count_Death_in_rows == 2:
            ddc_template = 'tbDDC2'    
        elif count_Death_in_rows == 3:
            ddc_template = 'tbDDC3'        
    else:
        ddc_template = "NA"
    #print(ddc_template)    
    
    shDDCTemplates.range('A2').value = ddc_template 
    shLists.range('I3').value = first_rc_containing_death
    shLists.range('I152').value = second_rc_containing_death
    shLists.range('I301').value = third_rc_containing_death    
        
    ddc_dict = {
        "first_rc": {"rc_no": first_rc_containing_death},
        "second_rc": {"rc_no": second_rc_containing_death},
        "third_rc": {"rc_no": third_rc_containing_death}
    }
    
    
    
    return count_Death_in_rows, columns_with_death, ddc_dict, has_income_protection, ddc_template


In [26]:
#3.1
def extract_inv_number(column_name):
    match = re.search(r'I/B(\d+):', column_name)
    if match:
        return match.group(1)
    return "NA"

#3.2
def closest_invmatch_df(s, df):
    best_match = None
    highest_score = 0
    for col in df.columns:
        score = fuzz.ratio(s, col)
        if score > highest_score:
            best_match = col
            highest_score = score
    return df.columns.get_loc(best_match) if highest_score > 80 else -1

#3.3
def no_inv_contracts(df_import_pc, currentwb, shInvestTemplates, shLists):
    df_ib_inv_contracts = df_import_pc.loc[:, df_import_pc.columns.str.contains('i/b', case=False) & df_import_pc.columns.str.contains('category', case=False)]
    df_ib_inv_contracts.head()
    keywords = ['pension', 'retirement', 'flex', 'endowment', 'living', 'life', 'private']
    columns_with_keywords = [column for column in df_ib_inv_contracts.columns if any(df_ib_inv_contracts[column].astype(str).str.contains('|'.join(keywords), case=False))]

    # Update count_ib_inv to reflect the correct count
    count_ib_inv = len(columns_with_keywords)

    if len(columns_with_keywords) > 0:
        first_ib_inv = re.search(r'I/B(\d+)', columns_with_keywords[0]).group(1)
    else:
        first_ib_inv = "NA"

    if len(columns_with_keywords) > 1:
        second_ib_inv = re.search(r'I/B(\d+)', columns_with_keywords[1]).group(1)
    else:
        second_ib_inv = "NA"

    if len(columns_with_keywords) > 2:
        third_ib_inv = re.search(r'I/B(\d+)', columns_with_keywords[2]).group(1)
    else:
        third_ib_inv = "NA"
    
    inv_dict = {
        "first_inv": {"inv_no": first_ib_inv},
        "second_inv": {"inv_no": second_ib_inv},
        "third_inv": {"inv_no": third_ib_inv}
    }

    # Compute max_fund_alct
    max_fund_alct = 0
    for inv in inv_dict.values():
        column_index = closest_invmatch_df('I/B' + inv['inv_no'] + ':_Fund_Alc_%', df_import_pc)
        if column_index != -1:  # Add this check to ensure we have a valid column index
            column_name = df_import_pc.columns[column_index]
            fund_alct_str = df_import_pc[column_name].iloc[0]
            if pd.isnull(fund_alct_str) or fund_alct_str == 0:
                fund_alct = 0
            else:
                fund_alct = fund_alct_str.count('%')
            max_fund_alct = max(max_fund_alct, fund_alct)

    # Determine inv_template
    if count_ib_inv == 0:
        inv_template = "NA"
    elif count_ib_inv in {1, 2, 3} and max_fund_alct in {0, 1, 2, 3}:
        inv_template = 'tbInvest' + str(count_ib_inv)
    else:
        inv_template = 'tbInvest3'

    shInvestTemplates.range('A2').value = inv_template
    shLists.range('I4').value = first_ib_inv
    shLists.range('I153').value = second_ib_inv
    shLists.range('I302').value = third_ib_inv
    
    return count_ib_inv

In [27]:
def calc_rc(df_import_pc):
    df_import_pc_categories = df_import_pc.loc[:, df_import_pc.columns.str.contains('rc', case=False) & df_import_pc.columns.str.contains('category', case=False)]
    
    rc_dict = {}
    rc_dict_merged = {}

    for i in range(1, 9):  # looping from 1 to 8
        key = f'{ordinal(i)}_rc'
        field_name = f'RC{i}: Category'
        
        closest_match_index = closest_match_df(field_name, df_import_pc_categories)
        closest_match_column = df_import_pc_categories.columns[closest_match_index]
        
        rc_no = i  # instead of extracting from closest_match_column, we use i
        
        category = df_import_pc_categories.iloc[0, closest_match_index]
        
        rc_dict[key] = {"field_name": field_name, "rc_no": rc_no, "category": category}
        
        # updating the category
        if category is not None:
            if 'Short Term' in category:
                category = 'st'
            elif any(term.lower() in category.lower() for term in ['medical', 'gap', 'health']):
                category = 'medgap'
            elif any(term.lower() in category.lower() for term in ['death', 'funeral', 'estate']):
                category = 'ddc'
        
        rc_dict_merged[key] = {"field_name": field_name, "rc_no": rc_no, "category": category}
    
    return rc_dict_merged


# Function to build dictionary
def build_dictionary(data_dict, category, prefix):
    return_dict = {}
    return_dict[f'{prefix}'] = {}
    count = 1
    for k, v in data_dict.items():
        if v['category'] == category:
            return_dict[f'{prefix}'][f'{ordinal(count)}_{prefix}'] = v['rc_no']
            count += 1
    return_dict[f'{prefix}'][f'no_{prefix}'] = count-1
    return return_dict


def update_df_field_list(df_tbFieldList, rc_ddc_dict, rc_st_dict, rc_medgap_dict, df_import_pc):
    df_tb_FieldList_1st_update = df_tbFieldList.copy()

    # Fill NaN values with 'TBC'
    df_tb_FieldList_1st_update['Calculated Field Name'].fillna('TBC', inplace=True)

    # iterating over each dictionary
    for dict_obj, dict_name in [(rc_ddc_dict, 'rc_ddc'), (rc_st_dict, 'rc_st'), (rc_medgap_dict, 'rc_medgap')]:
        for key, value in dict_obj[dict_name].items():
            if 'no_' not in key:  # exclude the 'no_' keys
                matching_rows = df_tb_FieldList_1st_update['Calculated Field Name'].str.contains('\(' + key + '\)')

                # loop through matching rows and update Calculated Field Name and Result
                for index, row in df_tb_FieldList_1st_update[matching_rows].iterrows():
                    # updating 'Calculated Field Name'
                    updated_field_name = row['Calculated Field Name'].replace('(' + key + ')', str(value))
                    df_tb_FieldList_1st_update.loc[index, 'Calculated Field Name'] = updated_field_name
                    
                    # finding closest match in df_import_pc
                    closest_match_index = closest_match_df(updated_field_name, df_import_pc)
                    closest_match_column = df_import_pc.columns[closest_match_index]
                    
                    # updating 'Result'
                    result_value = df_import_pc.iloc[0, closest_match_index]
                    if pd.notna(result_value) and result_value != 0:
                        df_tb_FieldList_1st_update.loc[index, 'Result'] = result_value
    return df_tb_FieldList_1st_update


In [28]:
#4 import df_calcs_pc

In [29]:
#5 Extract wp from ddc
def extract_and_manipulate_data(df):
    # Find the rows containing the search string in the Field Description column
    mask = df['Field Description'].str.contains('rc_wp_term', na=False)
    
    # Extract the corresponding values from the Result column
    result_values = df.loc[mask, 'Result'].values
    
    if len(result_values) > 0:
        # Extract wp1 and wp2 from the first matching value
        field_value = result_values[0]
        split_values = field_value.split('/', maxsplit=1)
        
        if len(split_values) > 1:
            wp1 = split_values[1].strip()
            wp2 = split_values[0].strip()
            
            # Update the Result column for the matching row
            df.loc[mask, 'Result'] = wp1
            
            # Insert wp2 in the Result column offset down 1 row
            df['Result'] = df['Result'].shift(1)
            df.loc[mask, 'Result'] = wp2
            
            # Insert wp2 in the Result column offset down 2 rows
            df['Result'] = df['Result'].shift(1)
            df.loc[mask, 'Result'] = wp2
    
    return df

In [30]:
#6 convert tbFieldsList to dict_tbFieldsList

In [31]:
#7 Find closest match btw first PC import and dict_tbFieldsList Return dict_tbFieldsList_updated
def update_dict_tbFieldsList(dict_tbFieldsList, df_import_pc):
    dict_tbFieldsList_updated = dict_tbFieldsList.copy()

    for item in dict_tbFieldsList_updated:
        calculated_field_name = item['Calculated Field Name']
        closest_match_index = closest_match_df(calculated_field_name, df_import_pc)

        if closest_match_index != -1:
            closest_match_ratio = fuzz.ratio(calculated_field_name, df_import_pc.columns[closest_match_index])
            if closest_match_ratio > 90:
                item['Result'] = df_import_pc.iloc[0, closest_match_index]
    
    return dict_tbFieldsList_updated

In [32]:
#8. extract calculations in ddc_dict
def extract_and_manipulate_data(df):
    # Find the rows containing the search string in the Field Description column
    mask = df['Field Description'].str.contains('rc_wp_term', na=False)
    
    # Extract the corresponding values from the Result column
    result_values = df.loc[mask, 'Result'].values
    
    if len(result_values) > 0:
        # Extract wp1 and wp2 from the first matching value
        field_value = result_values[0]
        
        # Check if the field_value is None before trying to split
        if field_value is not None:
            split_values = field_value.split('/', maxsplit=1)
        
            if len(split_values) > 1:
                wp1 = split_values[1].strip()
                wp2 = split_values[0].strip()
            
                # Update the Result column for the matching row
                df.loc[mask, 'Result'] = wp1
            
                # Insert wp2 in the Result column offset down 1 row
                df['Result'] = df['Result'].shift(1)
                df.loc[mask, 'Result'] = wp2
            
                # Insert wp2 in the Result column offset down 2 rows
                df['Result'] = df['Result'].shift(1)
                df.loc[mask, 'Result'] = wp2
    
    return df



In [33]:
def extract_key_variables(list_of_dicts):
    key_variables = ["First Name", "Last Name"]
    extracted_variables = {}

    for var in key_variables:
        closest_match, result = find_closest_match(var, list_of_dicts)  # Assign returned values to two variables
        if closest_match != 'Not Found':
            extracted_variables[var] = result
        else:
            extracted_variables[var] = 'Not Found'

    return extracted_variables


In [34]:
# Modifying your set_wrd_doc function to accept extracted_variables
def set_wrd_doc(current_datetime, parent_folder, closest_folder_full_path, extracted_variables):
    # Extracting First_Name and Last_Name from the extracted_variables dictionary
    First_Name = extracted_variables.get("First Name", "")
    Last_Name = extracted_variables.get("Last Name", "")

    # Set word template
    wrd_template = os.path.join(str(parent_folder), "snapshot2.docm")
    version = 1
    new_folder_name = f"{Last_Name}.{First_Name}.[{Ref_No}]"
    new_wrd_template_filepath = os.path.join(str(closest_folder_full_path), "snapshot" + current_datetime + "vs" + str(version) + ".docm")
    # Check if file already exists, if yes increment the version
    while os.path.exists(new_wrd_template_filepath):
        version += 1
        new_wrd_template_filepath = os.path.join(str(closest_folder_full_path), "snapshot" + current_datetime + "vs" + str(version) + ".docm")
    try:
        # Copy the word template to new location
        copyfile(wrd_template, new_wrd_template_filepath)
    except FileNotFoundError:
        #print("File not found. Please select the correct file.")
        root = tk.Tk()
        root.withdraw()
        wrd_template = filedialog.askopenfilename()
        copyfile(wrd_template, new_wrd_template_filepath)
    return new_wrd_template_filepath

In [35]:
#11.1 Convert Transposed Dict to sheet

#11.2 Write Transposed Dict to Sheet
def apply_vars(extracted_vars, currentwb, shCalcs):
    # Use values from extracted_vars to set values in shCalcs
    if 'First Name' in extracted_vars:
        shCalcs.range('A2').value = extracted_vars['First Name']
    else:
        print("First Name not found in extracted_vars")
        
    if 'Last Name' in extracted_vars:
        shCalcs.range('B2').value = extracted_vars['Last Name']
    else:
        print("Last Name not found in extracted_vars")

#11.3 Write Closest Folder to sheet

In [36]:
def set_wrd_doc(current_datetime, parent_folder, client_folder_path):
    # Set word template
    wrd_template = os.path.join(str(parent_folder), "snapshot.docm")
    version = 1
    new_wrd_template_filepath = os.path.join(str(client_folder_path), "snapshot" + current_datetime + "vs" + str(version) + ".docm")

    # Check if file already exists, if yes increment the version
    while os.path.exists(new_wrd_template_filepath):
        version += 1
        new_wrd_template_filepath = os.path.join(str(client_folder_path), "snapshot" + current_datetime + "vs" + str(version) + ".docm")


    return new_wrd_template_filepath

In [37]:
#11. Convert df to dict

def convert_df_to_dict(df, dict_tbFieldsList_updated, extracted_vars, currentwb, shLists, shCalcs, shStart):
    df_dict = df.to_dict(orient='records')

    values = [item['Result'] for item in dict_tbFieldsList_updated]
    transposed_values = np.transpose([values])
    shLists.range('M3').value = transposed_values
    apply_vars(extracted_vars, currentwb, shCalcs)
    Ref_No = shStart.range('A2').value

    return df_dict


In [38]:
#Main
def snapshot_primary():
    # 1: Create New Exc+Wrd Templates + Import workbooks / sheets / dataframes
    current_datetime, shUserMapping, parent_folder, shhdr_col_static, df_hdr_col_static, df_tb_calc_pc, df_import_pc, currentwb, shcalcs_pc, shCalcs, shLists, shDDCTemplates, shInvestTemplates, shSTTemplates, shMedGapTemplates, shStart, tbTemplList, tbFieldList, df_tbTemplList, df_tbFieldList = defineshtb()

    #1.2
    Ref_No, folder_prefix, user_word_template, folder_1_suffix, folder_2_suffix, folder_3_suffix = defineusermapping(shUserMapping, currentwb, shStart, shCalcs)
    
    # 2: Extract ddc data + write 1-8 ddc + ddc_templ to sheets
    count_Death_in_rows, columns_with_death, ddc_dict, has_income_protection, ddc_template = No_of_Contracts(df_import_pc, currentwb, shDDCTemplates, shLists)

    # 3.1.2: Extract inv data + write 1-8 inv + inv_templ to sheets
    #count_ib_inv, columns_with_keywords, inv_dict, max_fund_alct, inv_template = no_inv_contracts(df_import_pc, currentwb, shInvestTemplates, shLists)
    #print(count_ib_inv, columns_with_keywords, inv_dict, max_fund_alct, inv_template)

    # 4: Import Calc_pc + Create Calc_import_pc + convert to dict + replace all ":+ "
    df_calc_pc = shCalcs.range("A1").options(pd.DataFrame, expand="table").value

    # 5: Extract ddc calc data + write 1-8 ddc calc to sheets
    df_tbFieldList = extract_and_manipulate_data(df_tbFieldList)

    # 6: Update dict_tbFieldsList and remove text between brackets using regular expression in update_dict_tbFieldsList
    # In your main function replace step 6 with:

    # 6: Update dict_tbFieldsList and remove text between brackets using regular expression in update_dict_tbFieldsList
    df_import_pc_processed = remove_text_in_brackets(df_import_pc)

    # 7 Convert df_tbFieldList to dict
    dict_tbFieldsList = df_tbFieldList.to_dict('records')
    
    # 8: Get Unique_values are the keys in your dict_tbFieldsList_updated
    dict_tbFieldsList_updated = update_dict_tbFieldsList(dict_tbFieldsList, df_import_pc_processed)
    #print(dict_tbFieldsList_updated)

    # 9: Process ddc_dict
    processed_ddc_dict = process_ddc_dict(ddc_dict, df_import_pc_processed)
    #processed_ddc_dict) 
    
    # 10: Extract Key Variables
    extracted_variables = extract_key_variables(dict_tbFieldsList_updated)

    # 11: Convert df_import_pc to dictionary and perform additional tasks
    df_import_pc_dict = convert_df_to_dict(df_import_pc_processed, dict_tbFieldsList_updated, extracted_variables, currentwb, shLists, shCalcs, shStart)

    # Extract First_Name and Last_Name from extracted_variables
    First_Name = extracted_variables.get("First Name", "")
    Last_Name = extracted_variables.get("Last Name", "")
    
    # 12: Find or Create Folder
    client_folder_path = find_or_create_folder(Last_Name, First_Name, Ref_No, folder_prefix, folder_1_suffix, folder_2_suffix, folder_3_suffix)

    # 13: Set word template    
    new_wrd_template_filepath = set_wrd_doc(current_datetime, parent_folder, client_folder_path)
    # Set the string value of the word template filepath
    shCalcs.range('F2').value = new_wrd_template_filepath    


In [39]:
#Testing

def testing():
    # 1: Create New Exc+Wrd Templates + Import workbooks / sheets / dataframes
    current_datetime, shUserMapping, parent_folder, shhdr_col_static, df_hdr_col_static, df_tb_calc_pc, df_import_pc, currentwb, shcalcs_pc, shCalcs, shLists, shDDCTemplates, shInvestTemplates, shSTTemplates, shMedGapTemplates, shStart, tbTemplList, tbFieldList, df_tbTemplList, df_tbFieldList = defineshtb()
    #print(df_import_pc.head())  
    rc_dict_merged = calc_rc(df_import_pc)

    rc_ddc_dict = build_dictionary(rc_dict_merged, 'ddc', 'rc_ddc')
    rc_medgap_dict = build_dictionary(rc_dict_merged, 'medgap', 'rc_medgap')
    rc_st_dict = build_dictionary(rc_dict_merged, 'st', 'rc_st')
    
    print(rc_dict_merged)
    print(rc_ddc_dict)
    print(rc_medgap_dict)
    print(rc_st_dict)
    
    df_tb_FieldList_1st_update = update_df_field_list(df_tbFieldList, rc_ddc_dict, rc_st_dict, rc_medgap_dict, df_import_pc)
    print(df_tb_FieldList_1st_update)


In [40]:
testing()

{'1st_rc': {'field_name': 'RC1: Category', 'rc_no': 1, 'category': 'ddc'}, '2nd_rc': {'field_name': 'RC2: Category', 'rc_no': 2, 'category': 'ddc'}, '3rd_rc': {'field_name': 'RC3: Category', 'rc_no': 3, 'category': 'ddc'}, '4th_rc': {'field_name': 'RC4: Category', 'rc_no': 4, 'category': 'st'}, '5th_rc': {'field_name': 'RC5: Category', 'rc_no': 5, 'category': 'st'}, '6th_rc': {'field_name': 'RC6: Category', 'rc_no': 6, 'category': 'medgap'}, '7th_rc': {'field_name': 'RC7: Category', 'rc_no': 7, 'category': 'medgap'}, '8th_rc': {'field_name': 'RC8: Category', 'rc_no': 8, 'category': None}}
{'rc_ddc': {'1st_rc_ddc': 1, '2nd_rc_ddc': 2, '3rd_rc_ddc': 3, 'no_rc_ddc': 3}}
{'rc_medgap': {'1st_rc_medgap': 6, '2nd_rc_medgap': 7, 'no_rc_medgap': 2}}
{'rc_st': {'1st_rc_st': 4, '2nd_rc_st': 5, 'no_rc_st': 2}}
         2              Field Description  Calculated Field Name  Source  \
0      1.0             1st_rc_ddc_company            RC1:Company  Import   
1      1.0               1st_rc_ddc_de